<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Parsa Sharifi
### Student Number : 99101762
### Colab Link: https://colab.research.google.com/drive/17X2bxrBqn4jdjZhBWhJlw6PeIpurVtFx?usp=sharing
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [4]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-08 13:39:33--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-08 13:39:34--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc25753e29240abd01378e395c28.dl.dropboxusercontent.com/cd/0/get/B7p-IyvwxPz4WL5XTToeUThjMFA4P0uvGs9yL7I04_k0zd8T6AZrcV38tqYcMtGU0EqegdT73YC05q90i9BgprK0XiEpXNJ1Ef8ED9Wlsk7-H4nwTo2lxKJQ9G8KVw08-91s4rUg7fAk43HSkrpow4Pyuzoh5WBJJ_54ROuN0v-_0aeRYFo7H5EHFdY5Igq6OHA/file?dl=1# [following]
--2023-05-08 13:39:34--  https://uc25753e29240abd01378e395c28.dl.dropboxusercontent.com/cd/0/get/B7p-IyvwxPz4WL5XTToeUThjMFA4P0uvGs9yL7I04_

In [5]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

Archive:  /content/dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/dataset.zip or
        /content/dataset.zip.zip, and cannot find /content/dataset.zip.ZIP, period.


# Preprocessing

In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

I saved it in my google drive, if you want to run it again update the address

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
df = pd.read_csv('/content/drive/MyDrive/dataset/content/news-train.csv')
print(df.head())

       topic                                               text
0     sports  به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1   politics  پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2   politics  # دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3  economics  کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4  economics  به گزارش ایسنا صحبت با برخی # # آن است که امرو...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [9]:
topic_counts = df['topic'].value_counts()
topic_counts

sports       123105
economics     49600
cultural      13359
politics      11297
Name: topic, dtype: int64

Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [10]:
from sklearn.utils import resample

In [11]:
min_topic_count = topic_counts.min()

df_balanced = pd.DataFrame()
for topic in df['topic'].unique():
    df_topic = df[df['topic'] == topic]
    df_topic_downsampled = resample(df_topic, replace=False, n_samples=min_topic_count, random_state=42)
    df_balanced = pd.concat([df_balanced, df_topic_downsampled])


In [12]:
df_balanced['topic'].value_counts()

sports       11297
politics     11297
economics    11297
cultural     11297
Name: topic, dtype: int64

# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [13]:
train_df, test_df = train_test_split(df_balanced, test_size=0.2, random_state=42)


In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [14]:
tokens = train_df['text'].apply(lambda x: x.split())

print(tokens)

131394    [همه‌چیز, شبیه, مسابقه, هفته, بود, البته, بدون...
88791     [رییس, مجمع, تشخیص, مصلحت, نظام, با, اشاره, به...
121984    [مرحله, اول, مسابقات, لیگ, ورزش‌های, سه, گانه,...
12673     [#, بارها, تاکید, کرده‌اند, #, برنامه‌ای, برای...
106051    [به, گزارش, ورزش, سه, کمتر, کسی, فکر, می‌کرد, ...
                                ...                        
92968     [به, گزارش, ایسنا, جام‌جهانی, #, روسیه, #, روز...
169808    [نمایش, کمدی, #, به, کارگردانی, میثم, عبدی, از...
157638    [معاون, پژوهشی, جهاد, دانشگاهی, واحد, استان, ا...
118852    [به, گزارش, ورزش, سه, آبی, پوشان, پایتخت, که, ...
53822     [حسن, روحانی, روز, گذشته, در, بخشی, از, سخنان,...
Name: text, Length: 36150, dtype: object


Count the tokens and list the top 5 frequent ones (5 points)

In [16]:
from sklearn.feature_extraction.text import CountVectorizer

In [17]:
vectorizer = CountVectorizer()
vectorizer.fit(train_df['text'])
tokenized_text = vectorizer.transform(train_df['text'])

In [18]:
total = tokenized_text.sum(axis = 0)
number_of_tokens = [(token, total[0,index]) for token, index in vectorizer.vocabulary_.items()]
number_of_tokens = sorted(number_of_tokens, key = lambda x: x[1], reverse = True)

In [19]:
number_of_tokens[:5]

[('در', 743942),
 ('به', 609106),
 ('از', 435477),
 ('که', 425635),
 ('این', 424750)]

Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [ ]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

I saved it in my google drive, if you want to run it again update the address

In [20]:
with open('/content/drive/MyDrive/stopwords.txt', 'r', encoding='utf-8') as f:  
    stop_words = f.read().splitlines()
vectorizer = CountVectorizer(stop_words = stop_words)
vectorizer = vectorizer.fit(train_df['text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['اثر', 'البت', 'بالای', 'برابر', 'برای', 'بیرون', 'تول', 'توی', 'تی', 'جلوی', 'حدود', 'خارج', 'دنبال', 'روی', 'زیر', 'سری', 'سمت', 'سوی', 'ضد', 'طبق', 'عقب', 'عل', 'عنوان', 'قصد', 'لطفا', 'مد', 'نزد', 'نزدیک', 'وسط', 'پاعین', 'کنار'] not in stop_words.
  warnings.warn(


Make a list of the remaining unique tokens (5 points)

In [21]:
vectorizer.get_feature_names_out()

array(['آب', 'آبادان', 'آبان', ..., 'یکم', 'یکپارچه', 'یکی'], dtype=object)

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [22]:
train_x = vectorizer.transform(train_df['text']).toarray()

In [23]:
train_x

array([[1, 0, 0, ..., 0, 1, 3],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [24]:
train_y = train_df.topic.astype('category').cat.codes

In [25]:
train_y

131394    2
88791     2
121984    3
12673     2
106051    3
         ..
92968     3
169808    0
157638    0
118852    3
53822     2
Length: 36150, dtype: int8

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [26]:
test_x =  vectorizer.transform(test_df['text']).toarray()

In [27]:
test_x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 6],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [28]:
test_y = test_df.topic.astype('category').cat.codes

In [29]:
test_y

177683    2
158663    1
99757     0
39112     2
17310     2
         ..
150793    3
27165     1
161829    0
179421    3
34480     1
Length: 9038, dtype: int8

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [30]:
model = MultinomialNB()

In [31]:
model.fit(train_x, train_y)

MultinomialNB()

Get the predictions of the model for the testing news (`x_test`).

In [32]:
y_pred = model.predict(test_x)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [33]:
print(classification_report(test_y, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.95      0.95      2255
           1       0.91      0.94      0.93      2268
           2       0.94      0.90      0.92      2262
           3       0.99      0.98      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



## Using your own code (15 points)

In [38]:
import numpy as np

class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        """
        Initialize the classifier.
        """
        self.prior = None
        self.conditional = None

    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape

        self.classes_ = np.unique(y)
        self.priors = np.zeros(len(self.classes_))
        for i, c in enumerate(self.classes_):
            self.priors[i] = np.mean(y == c)

        self.counts = np.zeros((len(self.classes_), n_features), dtype=int)
        self.conditional = np.zeros((len(self.classes_), n_features))
        for i, c in enumerate(self.classes_):
            X_c = X[y == c]
            for j in range(n_features):
                self.counts[i, j] = np.sum(X_c[:, j])
                # Apply smoothing by adding one to the count and dividing by the total count plus the number of features
                self.conditional[i, j] = (self.counts[i, j] + 1) / (np.sum(self.counts[i]) + n_features)

        return self

    def predict(self, X):
      if not hasattr(self, "conditional"):
          raise ValueError("The model is not fitted yet. Call fit first.")

      n_samples, n_features = X.shape
      if n_features != self.conditional.shape[1]:
          raise ValueError(f"Expected {self.conditional.shape[1]} features, got {n_features}")

      log_probs = np.zeros((n_samples, len(self.classes_)))
      for i, c in enumerate(self.classes_):
          log_probs[:, i] = np.log(self.priors[i]) + np.sum(np.log(self.conditional[i]) * X, axis=1)

      return self.classes_[np.argmax(log_probs, axis=1)]


Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [39]:
my_model = Naive_Bayes()

In [40]:
my_model.fit(np.array(train_x),np.array(train_y))

Get the predictions of your model for the testing news (`x_test`).

In [41]:
my_model_pred = my_model.predict(np.array(test_x))

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [42]:
print(classification_report(test_y, my_model_pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94      2255
           1       0.91      0.95      0.93      2268
           2       0.95      0.90      0.92      2262
           3       0.99      0.99      0.99      2253

    accuracy                           0.95      9038
   macro avg       0.95      0.95      0.95      9038
weighted avg       0.95      0.95      0.95      9038



# Kaggle

Competition Link: https://www.kaggle.com/t/88f1b6e251e34575b2e4cb4b91aed0ef